In [1]:
import pandas_profiling 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

np.random.seed(546)

%matplotlib inline
plt.style.use('fivethirtyeight')
sns.set_style("dark")


#Criando nossa base de dados

bcdf = pd.read_csv('cancer_de_mama.csv')


## Criando uma cópia, caso seja necessário manter alguma das bases no estado original

bcdf_1 = bcdf.copy()

bcdf

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.30010,0.14710,...,17.33,184.60,2019.0,0.16220,0.66560,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.08690,0.07017,...,23.41,158.80,1956.0,0.12380,0.18660,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.19740,0.12790,...,25.53,152.50,1709.0,0.14440,0.42450,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.24140,0.10520,...,26.50,98.87,567.7,0.20980,0.86630,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.19800,0.10430,...,16.67,152.20,1575.0,0.13740,0.20500,0.4000,0.1625,0.2364,0.07678,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
564,926424,M,21.56,22.39,142.00,1479.0,0.11100,0.11590,0.24390,0.13890,...,26.40,166.10,2027.0,0.14100,0.21130,0.4107,0.2216,0.2060,0.07115,NaN
565,926682,M,20.13,28.25,131.20,1261.0,0.09780,0.10340,0.14400,0.09791,...,38.25,155.00,1731.0,0.11660,0.19220,0.3215,0.1628,0.2572,0.06637,NaN
566,926954,M,16.60,28.08,108.30,858.1,0.08455,0.10230,0.09251,0.05302,...,34.12,126.70,1124.0,0.11390,0.30940,0.3403,0.1418,0.2218,0.07820,NaN
567,927241,M,20.60,29.33,140.10,1265.0,0.11780,0.27700,0.35140,0.15200,...,39.42,184.60,1821.0,0.16500,0.86810,0.9387,0.2650,0.4087,0.12400,NaN


In [2]:
bcdf.rename(columns={'concave points_mean':'concave_points_mean'},inplace=True)
bcdf = bcdf.drop('Unnamed: 32', axis=1)

## Excluindo a coluna Unnamed:32

In [3]:
## Transformando variável categórica do diagnóstico em valores matemáticos
## No caso, os tecidos malignos são representados pelo número 1, e os benignos pelo número 0


bcdf['diagnosis'][bcdf['diagnosis']=='M'] = 1
bcdf['diagnosis'][bcdf['diagnosis']=='B'] = 0

C:\Users\João Vitor\AppData\Local\Temp\ipykernel_14888\1618916433.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bcdf['diagnosis'][bcdf['diagnosis']=='M'] = 1
C:\Users\João Vitor\AppData\Local\Temp\ipykernel_14888\1618916433.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bcdf['diagnosis'][bcdf['diagnosis']=='B'] = 0


In [4]:
bcdf['diagnosis'] = bcdf['diagnosis'].astype('int')

In [5]:
## Criando uma base de dados com os valores médios

bcdf.rename(columns={'concave points_mean':'concave_points_mean'},inplace=True)
mean_val = bcdf[['id', 'diagnosis', 'radius_mean', 'texture_mean', 'perimeter_mean',
       'area_mean', 'smoothness_mean', 'compactness_mean', 'concavity_mean',
       'concave_points_mean', 'symmetry_mean', 'fractal_dimension_mean']].copy()


mean_val_1 = mean_val.drop('id',axis=1)
mean_val_1 = mean_val_1.drop('diagnosis',axis=1)



No gráfico abaixo pretendemos estudar algumas interações a partir de gráficos tridimensionais.

- No primeiro, indivíduos com tumores malignos:
* Eixo X nos indicará a concavidade média
* Eixo Y a média da área
* Eixo Z teremos a simetria

- No segundo, indivíduos saudáveis

In [6]:
import plotly.graph_objs as go
import plotly.offline as py
from plotly.offline import init_notebook_mode, iplot

grph_1 = go.Scatter3d(
    x = bcdf[bcdf['diagnosis']==1].concavity_mean,
    y = bcdf[bcdf['diagnosis']==1].area_mean,
    z = bcdf[bcdf['diagnosis']==1].symmetry_mean,
    mode = 'markers',
    marker = dict(
         size = 10,
         color = mean_val_1['concavity_mean'],
         colorscale = 'YlGnBu'
    )
)

graph0 = [grph_1]

layout = go.Layout(margin=dict( l=0, r=0, b=0, t=0 ), scene = dict(
            xaxis = dict(title  = 'concavity_mean'),
            yaxis = dict(title  = 'area_mean'),
            zaxis = dict(title  = 'symmetry_mean')
        )
    
)
fig = go.Figure(data = graph0, layout=layout)
py.iplot(fig)

In [7]:
grph_1 = go.Scatter3d(
    x = bcdf[bcdf['diagnosis']==0].concavity_mean,
    y = bcdf[bcdf['diagnosis']==0].area_mean,
    z = bcdf[bcdf['diagnosis']==0].symmetry_mean,
    mode = 'markers',
    marker = dict(
         size = 10,
         color = mean_val_1['concavity_mean'],
         colorscale = 'YlGnBu'
    )
)

graph0 = [grph_1]

layout = go.Layout(margin=dict( l=0, r=0, b=0, t=0 ), scene = dict(
            xaxis = dict(title  = 'concavity_mean'),
            yaxis = dict(title  = 'area_mean'),
            zaxis = dict(title  = 'symmetry_mean')
        )
    
)
fig = go.Figure(data = graph0, layout=layout)
py.iplot(fig)